In [1]:
import sys; sys.path.append('..')
import cv2
import numpy as np
from scareddataset import iotools
from pathlib import Path
import scareddataset.data_maniputlation as dm
from scareddataset.calibrator import StereoCalibrator
from scareddataset import evaluation

In [2]:
scared_data_root_path = Path('/home/dimitrisps/new_data')
clean_scared_depthmap = scared_data_root_path / 'clean_left_depthmap'
clean_scared_depthmap_png = scared_data_root_path / 'clean_left_depthmap_png'
scared_depthmap_png = scared_data_root_path / 'depthmap_png'
scared_depthmap_recycled = scared_data_root_path / 'recycled_left_depthmap'
clean_scared_depthmap.mkdir(parents=True, exist_ok=True)
clean_scared_depthmap_png.mkdir(parents=True, exist_ok=True)
scared_depthmap_png.mkdir(parents=True, exist_ok=True)
scared_depthmap_recycled.mkdir(parents=True, exist_ok=True)

In [13]:
for clean_pc in (scared_data_root_path/ 'point_cloud_clean').iterdir():
    calib =StereoCalibrator().load(str(scared_data_root_path/ ('original_calib/' +clean_pc.stem+'.yaml')))
    pts3d = iotools.load_ply(str(clean_pc))
    depthmap, scared_depthmap = dm.pts3d_to_depthmap(pts3d, calib['K1'], calib['D1'], (1024,1280))
    iotools.save_subpix_png(clean_scared_depthmap_png/ (clean_pc.stem + '.png'), depthmap)
    iotools.save_depthmap_xyz(clean_scared_depthmap/ (clean_pc.stem + '.tiff'), scared_depthmap)



In [22]:
for gt in (scared_data_root_path/ 'left_depthmap').iterdir():
    calib =StereoCalibrator().load(str(scared_data_root_path/ ('original_calib/' +gt.stem+'.yaml')))
    pts3d = iotools.load_depthmap_xyz(gt).reshape(-1,3)
    depthmap, scared_depthmap = dm.pts3d_to_depthmap(pts3d, calib['K1'], calib['D1'], (1024,1280))
    iotools.save_subpix_png(scared_depthmap_png/ (gt.stem + '.png'), depthmap)
    iotools.save_depthmap_xyz(scared_depthmap_recycled/ (gt.stem + '.tiff'), scared_depthmap)



evaluate error between cleaned and original tiff depthmaps

In [12]:
original_gt = sorted([str(path) for path in (scared_data_root_path/'left_depthmap').iterdir() ])
clean_gt = sorted([str(path) for path in (scared_data_root_path/'clean_left_depthmap').iterdir() ])
for original_path, clean_path in zip( original_gt, clean_gt):
    fid = Path(original_path).stem
    original = iotools.load_depthmap_xyz(original_path)
    clean = iotools.load_depthmap_xyz(clean_path)
    results = evaluation.xyz_error(original, clean)
    print(fid, results)

1_1 (0.21326745, 0.7837417602539063, 0.55032958984375)
1_2 (0.2638105, 0.8304611206054687, 0.6294532775878906)
1_3 (0.2403629, 0.8681564331054688, 0.6576278686523438)
1_4 (0.086446725, 0.7297950744628906, 0.5398582458496094)
1_5 (0.191571, 0.719012451171875, 0.5088981628417969)
2_1 (0.09012184, 0.6527061462402344, 0.4771446228027344)
2_2 (0.2541776, 0.8476577758789062, 0.6655830383300781)
2_3 (0.14974071, 0.5437171936035157, 0.3793525695800781)
2_4 (0.0486424, 0.6760147094726563, 0.5083091735839844)
2_5 (0.3175981, 0.7500335693359375, 0.5631744384765625)
3_1 (0.31712297, 0.7493560791015625, 0.565252685546875)
3_2 (0.19515355, 0.7737480163574219, 0.5852485656738281)
3_3 (0.30127287, 0.7049179077148438, 0.49687881469726564)
3_4 (0.18330462, 0.7672004699707031, 0.5522323608398437)
3_5 (0.33067468, 0.6624839782714844, 0.47533416748046875)
4_1 (1.0771314, 0.7245307922363281, 0.5069389343261719)
4_2 (0.83283114, 0.8215568542480469, 0.6281532287597656)
4_3 (0.73911184, 0.7850860595703125, 0.5

In [5]:
original_gt = sorted([str(path) for path in (scared_data_root_path/'left_depthmap').iterdir() ])
recycled_gt = sorted([str(path) for path in (scared_data_root_path/'recycled_left_depthmap').iterdir() ])

for original_path, recycled_path in zip( original_gt, recycled_gt):
    fid = Path(original_path).stem
    original = iotools.load_depthmap_xyz(original_path)
    recycled = iotools.load_depthmap_xyz(recycled_path)
    results = evaluation.xyz_error(original, recycled)
    print(fid, results)

1_1 (0.21356557, 0.7837417602539063, 0.550439453125)
1_2 (0.26393092, 0.8304611206054687, 0.6295219421386719)
1_3 (0.2494799, 0.8681564331054688, 0.6576461791992188)
1_4 (0.08675561, 0.7297950744628906, 0.5400390625)
1_5 (0.19263753, 0.719012451171875, 0.5091445922851563)
2_1 (0.0924276, 0.6527061462402344, 0.47774581909179686)
2_2 (0.2590721, 0.8476577758789062, 0.6663787841796875)
2_3 (0.15170953, 0.5437171936035157, 0.3794769287109375)
2_4 (0.049141183, 0.6760147094726563, 0.5084434509277344)
2_5 (0.3184689, 0.7500335693359375, 0.563238525390625)
3_1 (0.31799233, 0.7493560791015625, 0.5655990600585937)
3_2 (0.19872664, 0.7737480163574219, 0.5852813720703125)
3_3 (0.3019838, 0.7049179077148438, 0.49692840576171876)
3_4 (0.18472253, 0.7672004699707031, 0.5523262023925781)
3_5 (0.33236018, 0.6624839782714844, 0.47535400390625)
4_1 (1.1626462, 0.7245307922363281, 0.5071342468261719)
4_2 (0.8428001, 0.8215568542480469, 0.6282066345214844)
4_3 (0.7507717, 0.7850860595703125, 0.53780975341

In [5]:
interpolated_clean_depthmaps_dir= Path(scared_data_root_path / 'interpolated_clean_left_depthmap')
interpolated_clean_depthmaps_png_dir =Path(scared_data_root_path / 'interpolated_clean_left_depthmap_png')

final_depthmaps_dir= Path(scared_data_root_path / 'final_left_depthmap')
final_depthmaps_png_dir =Path(scared_data_root_path / 'final_left_depthmap_png')


interpolated_clean_depthmaps_dir.mkdir(parents=True, exist_ok=True)
interpolated_clean_depthmaps_png_dir.mkdir(parents=True, exist_ok=True)

final_depthmaps_dir.mkdir(parents=True, exist_ok=True)
final_depthmaps_png_dir.mkdir(parents=True, exist_ok=True)

In [6]:
for clean_depthmap_png_path in clean_scared_depthmap_png.iterdir():
    calib =StereoCalibrator().load(str(scared_data_root_path/ ('original_calib/' +clean_depthmap_png_path.stem+'.yaml')))
    interpolated_depthmap_png = dm.interpolate_missing_1ch(iotools.load_subpix_png(clean_depthmap_png_path)[0])
    interpolated_scared_depthmaps = dm.depthmap_to_pts3d(interpolated_depthmap_png, calib['K1'], calib['D1'])
    iotools.save_subpix_png(interpolated_clean_depthmaps_png_dir/clean_depthmap_png_path.name, interpolated_depthmap_png)
    iotools.save_depthmap_xyz(interpolated_clean_depthmaps_dir/(clean_depthmap_png_path.stem +'.tiff'), interpolated_scared_depthmaps)


In [20]:
for interpolated_depthmap_png_path in interpolated_clean_depthmaps_png_dir.iterdir():
    calib =StereoCalibrator().load(str(scared_data_root_path/ ('original_calib/' +interpolated_depthmap_png_path.stem+'.yaml')))
    interpolated_depthmap = iotools.load_subpix_png(interpolated_depthmap_png_path)[0]
    smothed_depthmap =  cv2.GaussianBlur(interpolated_depthmap.astype(np.float32), (51,51), 10)
    scared_depthmap = dm.depthmap_to_pts3d(smothed_depthmap, calib['K1'], calib['D1'])
    iotools.save_depthmap_xyz(final_depthmaps_dir/(interpolated_depthmap_png_path.stem +'.tiff'), scared_depthmap)
    iotools.save_subpix_png(final_depthmaps_png_dir/interpolated_depthmap_png_path.name, smothed_depthmap)

In [22]:
original_gt = sorted([str(path) for path in (scared_data_root_path/'left_depthmap').iterdir() ])
final_gt = sorted([str(path) for path in (scared_data_root_path/'final_left_depthmap').iterdir() ])

for original_path, final_gt in zip( original_gt, final_gt):
    fid = Path(original_path).stem
    original = iotools.load_depthmap_xyz(original_path)
    final = iotools.load_depthmap_xyz(final_gt)
    results = evaluation.xyz_error(original, final)
    print(fid, results)

1_1 (0.8031849, 0.7837417602539063, 0.7837417602539063)
1_2 (0.9638647, 0.8304611206054687, 0.8304611206054687)
1_3 (0.94294506, 0.8681564331054688, 0.8681549072265625)
1_4 (0.50951207, 0.7297950744628906, 0.7297950744628906)
1_5 (0.68778205, 0.719012451171875, 0.719012451171875)
2_1 (0.5860685, 0.6527061462402344, 0.65247802734375)
2_2 (0.84406924, 0.8476577758789062, 0.8476287841796875)
2_3 (0.7235311, 0.5437171936035157, 0.5437171936035157)
2_4 (0.39645794, 0.6760147094726563, 0.6760147094726563)
2_5 (1.2630768, 0.7500335693359375, 0.7500335693359375)
3_1 (1.1858037, 0.7493560791015625, 0.7493560791015625)
3_2 (0.8942355, 0.7737480163574219, 0.7737480163574219)
3_3 (1.1070095, 0.7049179077148438, 0.7049179077148438)
3_4 (0.9188016, 0.7672004699707031, 0.7672004699707031)
3_5 (1.0359567, 0.6624839782714844, 0.6624839782714844)
4_1 (1.9511042, 0.7245307922363281, 0.7245307922363281)
4_2 (2.0921278, 0.8215568542480469, 0.8215568542480469)
4_3 (1.5883245, 0.7850860595703125, 0.785086059

In [11]:
final_semi_dense_depthmaps_png_dir =Path(scared_data_root_path / 'final_sparse_left_depthmap_png')
final_semi_dense_depthmaps_dir =Path(scared_data_root_path / 'final_sparse_left_depthmap')

final_semi_dense_depthmaps_png_dir.mkdir(parents=True, exist_ok=True)
final_semi_dense_depthmaps_dir.mkdir(parents=True, exist_ok=True)

In [12]:
for clean_depthmap_png_path in clean_scared_depthmap_png.iterdir():
    calib =StereoCalibrator().load(str(scared_data_root_path/ ('original_calib/' +clean_depthmap_png_path.stem+'.yaml')))
    clean = iotools.load_subpix_png(clean_depthmap_png_path)[0]
    final = iotools.load_subpix_png(final_depthmaps_png_dir/clean_depthmap_png_path.name)[0]
    final_semi_dense = final.copy()
    final_semi_dense[clean==0]=0
    final_semi_dense_scared = dm.depthmap_to_pts3d(final_semi_dense,calib['K1'], calib['D1'])
    iotools.save_subpix_png(final_semi_dense_depthmaps_png_dir/clean_depthmap_png_path.name, final_semi_dense)
    iotools.save_depthmap_xyz(final_semi_dense_depthmaps_dir/(clean_depthmap_png_path.stem +'.tiff'), final_semi_dense_scared)


In [13]:
original_gt = sorted([str(path) for path in (scared_data_root_path/'left_depthmap').iterdir() ])
final_sparse_gt = sorted([str(path) for path in (scared_data_root_path/'final_sparse_left_depthmap').iterdir() ])

for original_path, final_gt in zip( original_gt, final_sparse_gt):
    fid = Path(original_path).stem
    original = iotools.load_depthmap_xyz(original_path)
    final = iotools.load_depthmap_xyz(final_gt)
    results = evaluation.xyz_error(original, final)
    print(fid, results)

1_1 (0.7918856, 0.7837417602539063, 0.55032958984375)
1_2 (0.93791753, 0.8304611206054687, 0.6294532775878906)
1_3 (0.9122211, 0.8681564331054688, 0.6576278686523438)
1_4 (0.4880914, 0.7297950744628906, 0.5398582458496094)
1_5 (0.66345423, 0.719012451171875, 0.5088981628417969)
2_1 (0.5364626, 0.6527061462402344, 0.4771446228027344)
2_2 (0.7546803, 0.8476577758789062, 0.6655830383300781)
2_3 (0.6803394, 0.5437171936035157, 0.3793525695800781)
2_4 (0.3588982, 0.6760147094726563, 0.5083091735839844)
2_5 (1.2456032, 0.7500335693359375, 0.5631744384765625)
3_1 (1.1397939, 0.7493560791015625, 0.565252685546875)
3_2 (0.84761184, 0.7737480163574219, 0.5852485656738281)
3_3 (1.0943161, 0.7049179077148438, 0.49687881469726564)
3_4 (0.86726105, 0.7672004699707031, 0.5522323608398437)
3_5 (1.0079345, 0.6624839782714844, 0.47533416748046875)
4_1 (1.2609231, 0.7245307922363281, 0.5069389343261719)
4_2 (1.1836874, 0.8215568542480469, 0.6281532287597656)
4_3 (0.95631593, 0.7850860595703125, 0.5373764

# Disparities

Create disparities with both full and sparce ground truth with stereo rectification value = 0


In [17]:
comple_calibration_dir = Path(scared_data_root_path / 'complete_calib')
rect_left_dir = Path(scared_data_root_path / 'rect_left')
rect_right_dir = Path(scared_data_root_path / 'rect_right')

disparity_sparse_dir = Path(scared_data_root_path / 'disparity_sparse')
disparity_dir = Path(scared_data_root_path / 'disparity')

comple_calibration_dir.mkdir(parents=True, exist_ok=True)
rect_left_dir.mkdir(parents=True, exist_ok=True)
rect_right_dir.mkdir(parents=True, exist_ok=True)
disparity_sparse_dir.mkdir(parents=True, exist_ok=True)
disparity_dir.mkdir(parents=True, exist_ok=True)

In [28]:
dense_gt = sorted([str(path) for path in (scared_data_root_path/'final_left_depthmap').iterdir() ])
sparse_gt = sorted([str(path) for path in (scared_data_root_path/'final_sparse_left_depthmap').iterdir() ])
calib_init = sorted([str(path) for path in (scared_data_root_path/'original_calib').iterdir() ])
left_dir = (scared_data_root_path/'left')
right_dir = (scared_data_root_path/'right')
for dense_tiff_path, sparse_tiff_path, calib_path in zip(dense_gt, sparse_gt, calib_init):
    calibrator = StereoCalibrator()
    calibrator.load(calib_path)

    left=cv2.imread(str(left_dir/(Path(dense_tiff_path).stem+'.png')))
    right=cv2.imread(str(right_dir/(Path(dense_tiff_path).stem+'.png')))

    left_rect, right_rect = calibrator.rectify(left, right, alpha=0)

    cv2.imwrite(str(rect_left_dir/(Path(dense_tiff_path).stem+'.png')), left_rect)
    cv2.imwrite(str(rect_right_dir/(Path(dense_tiff_path).stem+'.png')), right_rect)

    calibrator.save(str(comple_calibration_dir/(Path(dense_tiff_path).stem+'.json')))
    
    dense = iotools.load_depthmap_xyz(dense_tiff_path)
    sparse = iotools.load_depthmap_xyz(sparse_tiff_path)
    calib = calibrator.calib

    dense_rot = dm.transform_pts(dense.reshape(-1,3), dm.create_RT(R=calib['R1']))
    sparse_rot = dm.transform_pts(sparse.reshape(-1,3), dm.create_RT(R=calib['R1']))

    disparity_dense = dm.ptd3d_to_disparity(dense_rot, calib['P1'],calib['P2'], (1024,1280))
    disparity_sparse = dm.ptd3d_to_disparity(sparse_rot, calib['P1'],calib['P2'], (1024,1280))

    iotools.save_subpix_png(disparity_sparse_dir/(Path(dense_tiff_path).stem+'.png'), disparity_sparse)
    iotools.save_subpix_png(disparity_dir/(Path(dense_tiff_path).stem+'.png'), disparity_dense)


In [25]:
calib_init

[]